# Reference

YouTube - Leila Gharani - Convert Columns to Rows in Excel (NO Transpose, NO Formulas - SIMPLY UNPIVOT in Power Query)

https://youtu.be/JMrfbv2h7p8

## Example

In [1]:
import pandas as pd

xl_file = 'inputs/Power_Query_Transpose_Replace_XelPlus.xlsx'
df = pd.read_excel(xl_file, sheet_name='DataTable', skiprows=2).fillna(0)

# Ensure numeric values are correctly 'typed'
df.loc[:, 'Jan':'Dec'] = df.loc[:, 'Jan':'Dec'].astype(int)

# Set indexing
df = df.set_index(['App', 'Type'])
df

Jan  Feb  Mar  Apr  May   Jun   Jul   Aug   Sep   Oct  \
App       Type                                                             
WenCaL    Sales     22   32   44   54   68    76    89   110   113   191   
          Volume     0    0    0   54   68    76    89   110   113     0   
WenCal    Returns    0    0   22    0    0     0     0     0     0     0   
Blend     Sales    149  274  490  641  823  1003  1164  1272  1395  1538   
Voltage   Volume    47  144  268  358  430   516   608   710   796   866   
Inkly     Returns  119  139  245  350  410   499   547   520   550   551   
Right App Sales    146  307  433  560  669   780   884  1263  1263  1263   
Kind Ape  Volume    42   63  128  142  214   316   419   491   576   644   
Pet Feed  Returns   69  194  401  668  864  1083  1363  1364  1483  1720   

                    Nov   Dec  
App       Type                 
WenCaL    Sales     276   264  
          Volume      0     0  
WenCal    Returns     0     0  
Blend     Sales    1712  1750  
Voltage   Volume    990  1000  
Inkly     Returns   573   472  
Right App Sales    1265  1275  
Kind Ape  Volume    650   551  
Pet Feed  Returns  1909  1913

## Wide to long format

In [2]:
df_melt = df.reset_index().melt(id_vars=['App', 'Type'])
df_melt.head()

,App,Type,variable,value
0,WenCaL,Sales,Jan,22
1,WenCaL,Volume,Jan,0
2,WenCal,Returns,Jan,0
3,Blend,Sales,Jan,149
4,Voltage,Volume,Jan,47


## Long to wide format

In [3]:
from piper import piper
from piper.verbs import *

piper version 0.0.7, last run: Friday, 26 February 2021 15:06:28


### Issue: months reported in wrong order

In [4]:
%%piper

df_melt 
>> pivot_table(index=['App', 'Type'], columns='variable', values='value')

variable           Apr   Aug   Dec  Feb  Jan   Jul   Jun  Mar  May   Nov  \
App       Type                                                             
Blend     Sales    641  1272  1750  274  149  1164  1003  490  823  1712   
Inkly     Returns  350   520   472  139  119   547   499  245  410   573   
Kind Ape  Volume   142   491   551   63   42   419   316  128  214   650   
Pet Feed  Returns  668  1364  1913  194   69  1363  1083  401  864  1909   
Right App Sales    560  1263  1275  307  146   884   780  433  669  1265   
Voltage   Volume   358   710  1000  144   47   608   516  268  430   990   
WenCaL    Sales     54   110   264   32   22    89    76   44   68   276   
          Volume    54   110     0    0    0    89    76    0   68     0   
WenCal    Returns    0     0     0    0    0     0     0   22    0     0   

variable            Oct   Sep  
App       Type                 
Blend     Sales    1538  1395  
Inkly     Returns   551   550  
Kind Ape  Volume    644   576  
Pet Feed  Returns  1720  1483  
Right App Sales    1263  1263  
Voltage   Volume    866   796  
WenCaL    Sales     191   113  
          Volume      0   113  
WenCal    Returns     0     0

### Fixed: monthly summary reporting (including totals)

Define months to categorical values

In [5]:
months = df_melt.variable.unique().tolist()
cat_months = pd.CategoricalDtype(categories=months, ordered=True)

In [6]:
%%piper

df_melt 
# >> where("variable.isin(['Jan', 'Feb', 'Mar', 'Apr', 'May'])")
>> assign(variable=lambda x: x.variable.astype(dtype=cat_months))
>> pivot_table(index=['App', 'Type'], columns='variable', values='value')
>> adorn(axis='both', ignore_row_index=False)

Jan   Feb   Mar   Apr   May   Jun   Jul   Aug   Sep   Oct  \
App       Type                                                                 
Blend     Sales    149   274   490   641   823  1003  1164  1272  1395  1538   
Inkly     Returns  119   139   245   350   410   499   547   520   550   551   
Kind Ape  Volume    42    63   128   142   214   316   419   491   576   644   
Pet Feed  Returns   69   194   401   668   864  1083  1363  1364  1483  1720   
Right App Sales    146   307   433   560   669   780   884  1263  1263  1263   
Voltage   Volume    47   144   268   358   430   516   608   710   796   866   
WenCaL    Sales     22    32    44    54    68    76    89   110   113   191   
          Volume     0     0     0    54    68    76    89   110   113     0   
WenCal    Returns    0     0    22     0     0     0     0     0     0     0   
          All      594  1153  2031  2827  3546  4349  5163  5840  6289  6773   

                    Nov   Dec    All  
App       Type                        
Blend     Sales    1712  1750  12211  
Inkly     Returns   573   472   4975  
Kind Ape  Volume    650   551   4236  
Pet Feed  Returns  1909  1913  13031  
Right App Sales    1265  1275  10108  
Voltage   Volume    990  1000   6733  
WenCaL    Sales     276   264   1339  
          Volume      0     0    510  
WenCal    Returns     0     0     22  
          All      7375  7225  53165